#Modules

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import h5py
import math


from sklearn.metrics import f1_score

from collections import deque

FMRI_DIR_STND = 'C:/Users/007303173/Documents/nsd_data/standardized-betas/subj01'
PATH = 'C:/Users/007303173/Documents/nsd_data/saved_models/subj01' #path for checkpoint

#Dataset Class

In [3]:
class DatasetNSD(Dataset):
  def __init__(self):
      #fmri_files will store all the betas files in the given directory
      self.dir = FMRI_DIR_STND
      self.fmri_files = self.getDirFiles()
      # num_of_scans is set by getRanges
      self.num_of_scans = 0
      # index_ranges is a dictionery with index range as key and corresponding file as value
      self.index_ranges =  self.getRanges()
      self.file_handlers = {}
      self.open_files()

  def __del__(self):
    for key, value in self.file_handlers.items():
      value.close()


  def __len__(self):
    return self.num_of_scans 

#Input: hdf5 file name 
#Output: store hdf5 in deque, return file object
  def open_files(self): 
    for file_name in self.fmri_files:
      path = os.path.join(FMRI_DIR_STND, file_name)
      self.file_handlers[file_name] = h5py.File(path, 'r')

   
  def __getitem__(self, index):
    #index will be 0-17908
    # files come in diffirent sizes of fmri scans.
    # a file will be chose based on the range of the index 
    for key in self.index_ranges:
      index_range = list(key)
      if (index in index_range):
        f = self.file_handlers[self.index_ranges[key]]
        indx = index - f['startIndx/i'][0]
        sample = torch.FloatTensor(np.array(f['betas/b'][indx]))
        label = torch.FloatTensor(np.array(f['labels/l'][indx]))
        break
    return (sample, label)

     
  def getDirFiles(self):
     files = [f for f in os.listdir(self.dir) if 
              os.path.isfile(os.path.join(self.dir, f)) and
              f[-5:] == '.hdf5']

     files.sort()                          
     return files

  def getRanges(self):
    ranges = {}
    previous = 0
    count = 0
    for x in self.fmri_files:
      with h5py.File(os.path.join(self.dir, x), "r") as f:    
          size = f['labels/l'].shape[0]
          count = count + size
          if previous == 0:
            ranges[range(size)] = x
            previous = size
          else:
            size = size + previous
            i = range(previous,size)
            ranges[i] = x
            previous = size
    self.num_of_scans = count
    return ranges

#NeuralNet3

In [3]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.kernel = 2
        self.stride = 2 
        self.conv_stack = nn.Sequential(
        nn.Conv3d(1,out_channels=64 , kernel_size = 3, stride = 3),
        nn.LeakyReLU(),
        nn.Conv3d(in_channels=64,out_channels=32 , kernel_size = self.kernel, stride = self.stride),
        nn.LeakyReLU(),
        nn.Conv3d(in_channels=32,out_channels=16 , kernel_size = self.kernel, stride = self.stride),
        nn.LeakyReLU(),
        nn.Flatten()
        )
        self.linear_stack = nn.Sequential(
        nn.Linear(4608,2000),
        nn.ReLU(),
        nn.Dropout(p=0.2),
        nn.Linear(2000,1000),
        nn.ReLU(),
        nn.Dropout(p=0.2),
        nn.Linear(1000,500),
        nn.ReLU(),
        nn.Dropout(p=0.2),
        nn.Linear(500,256),
        nn.ReLU(),
        nn.Linear(256,128),
        nn.ReLU(),
        nn.Linear(128,64),
        nn.ReLU(),
        nn.Linear(64, 1),
          )

    # # [(input_volume−kernel_size)/stride]+1
    # def output_size(self,volume):
    #   size = ((volume -self.kernel)//self.stride) + 1
    #   return size

    def forward(self, x):
        x = self.conv_stack(x.unsqueeze(dim=1))
        x = self.linear_stack(x)
        return x

#DataLoaders

In [ ]:
BATCH_SIZE = 199
ds = DatasetNSD()
split = len(ds)//6
split = len(ds)-split
#split =14925
train_dataset = Subset(ds, range(len(ds))[: split] )
print(len(train_dataset))
test_dataset = Subset(ds, range(len(ds))[split: ])
print(len(test_dataset))
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

14921
2984


#Initialize NeuralNet

In [ ]:
#device config
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

#hyper parameter
EPOCHS = 20
LEARNING_RATE = 0.0001
WEIGHT_DECAY = 0.01

modelNN = CNN().to(device)
# modelNN.load_state_dict(torch.load(os.path.join(PATH,'model_weights.pth')))
# weight = torch.FloatTensor([.322]).to(device)
loss_func = nn.BCEWithLogitsLoss()
criterion = torch.optim.Adam(modelNN.parameters(), lr = LEARNING_RATE)

print(modelNN)

Using cuda device
CNN(
  (conv_stack): Sequential(
    (0): Conv3d(1, 64, kernel_size=(3, 3, 3), stride=(3, 3, 3))
    (1): LeakyReLU(negative_slope=0.01)
    (2): Conv3d(64, 32, kernel_size=(2, 2, 2), stride=(2, 2, 2))
    (3): LeakyReLU(negative_slope=0.01)
    (4): Conv3d(32, 16, kernel_size=(2, 2, 2), stride=(2, 2, 2))
    (5): LeakyReLU(negative_slope=0.01)
    (6): Flatten(start_dim=1, end_dim=-1)
  )
  (linear_stack): Sequential(
    (0): Linear(in_features=4608, out_features=2000, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=2000, out_features=1000, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=1000, out_features=500, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.2, inplace=False)
    (9): Linear(in_features=500, out_features=256, bias=True)
    (10): ReLU()
    (11): Linear(in_features=256, out_features=128, bias=True)
    (12): ReLU()
    (13): Linear(in_features=128, out_featur

#Training Loop

In [ ]:
from tqdm import tqdm
def trainingLoop(train_ld, model, loss_function, optimizer , test_ld):
  scaler = GradScaler()
  trues = []
  preds = []
  for epoch in range(EPOCHS):
    model.train()
    avg_loss = 0
    n_total_steps = len(train_ld)
    for i, (sample, true_label) in enumerate(train_ld):
      sample = sample.to(device)
      trues.extend(true_label.numpy())
      true_label = true_label.to(device).unsqueeze(dim=1)
      pred_label = model(sample)
      loss = loss_function(pred_label, true_label)
      train_loss =  loss.item()
      avg_loss += train_loss
      #backward pass
      optimizer.zero_grad()
      scaler.scale(loss).backward()
      scaler.step(optimizer)
      scaler.update()
      #predictioins
      pred = torch.sigmoid(pred_label).detach().cpu()
      preds.extend(np.round(pred.reshape(-1)).numpy())
      if (i+1) % 5 == 0:
        print(f'epoch {epoch+1}/{EPOCHS},step {i+1}/{n_total_steps}, loss = {(train_loss):.4f} ')   
    score = f1_score(trues, preds)
    print(f'F1 score for training set is: {score}')
    torch.save(model.state_dict(), os.path.join(PATH,'model_weights.pth'))
    print(f'Average train loss for epoch: {avg_loss/n_total_steps} ')
    predTesting(test_ld, model)

def predTesting(loader, model ): 
  model.eval()
  with torch.no_grad():
    trues = []
    preds = []
    for  sample, true_label in tqdm(loader):
      sample = sample.to(device)
      trues.extend(true_label.numpy())
      true_label = true_label.to(device).unsqueeze(dim=1)
      pred_label = model(sample)
      pred = torch.sigmoid(pred_label).cpu()
      preds.extend(np.round(pred.reshape(-1)).numpy())
    score = f1_score(trues, preds)
    print(f'F1 score for validation set is: {score}')

In [ ]:
predTesting(test_loader, modelNN)

100%|██████████| 15/15 [01:35<00:00,  6.37s/it]

F1 score for training set is: 0.8692686623721106


In [ ]:
trainingLoop(train_loader, modelNN, loss_func, criterion, test_loader)

epoch 1/20,step 5/75, loss = 0.7148 
epoch 1/20,step 10/75, loss = 0.7046 
epoch 1/20,step 15/75, loss = 0.6740 
epoch 1/20,step 20/75, loss = 0.5975 
epoch 1/20,step 25/75, loss = 0.6832 
epoch 1/20,step 30/75, loss = 0.5003 
epoch 1/20,step 35/75, loss = 0.5886 
epoch 1/20,step 40/75, loss = 0.5432 
epoch 1/20,step 45/75, loss = 0.5350 
epoch 1/20,step 50/75, loss = 0.6557 
epoch 1/20,step 55/75, loss = 0.5083 
epoch 1/20,step 60/75, loss = 0.6119 
epoch 1/20,step 65/75, loss = 0.5412 
epoch 1/20,step 70/75, loss = 0.5410 
epoch 1/20,step 75/75, loss = 0.5551 
F1 score for training set is: 0.7917440486240082
Average train loss for epoch: 0.5972925515969595 


100%|██████████| 15/15 [03:05<00:00, 12.39s/it]


F1 score for validation set is: 0.8692686623721106
epoch 2/20,step 5/75, loss = 0.5635 
epoch 2/20,step 10/75, loss = 0.5773 
epoch 2/20,step 15/75, loss = 0.6005 
epoch 2/20,step 20/75, loss = 0.5822 
epoch 2/20,step 25/75, loss = 0.5704 
epoch 2/20,step 30/75, loss = 0.6153 
epoch 2/20,step 35/75, loss = 0.5635 
epoch 2/20,step 40/75, loss = 0.6061 
epoch 2/20,step 45/75, loss = 0.5858 
epoch 2/20,step 50/75, loss = 0.4883 
epoch 2/20,step 55/75, loss = 0.4799 
epoch 2/20,step 60/75, loss = 0.5433 
epoch 2/20,step 65/75, loss = 0.5712 
epoch 2/20,step 70/75, loss = 0.5827 
epoch 2/20,step 75/75, loss = 0.5278 
F1 score for training set is: 0.8273801167244941
Average train loss for epoch: 0.5543103003501892 


100%|██████████| 15/15 [03:05<00:00, 12.35s/it]


F1 score for validation set is: 0.8692686623721106
epoch 3/20,step 5/75, loss = 0.6292 
epoch 3/20,step 10/75, loss = 0.5163 
epoch 3/20,step 15/75, loss = 0.5128 
epoch 3/20,step 20/75, loss = 0.6133 
epoch 3/20,step 25/75, loss = 0.5579 
epoch 3/20,step 30/75, loss = 0.4788 
epoch 3/20,step 35/75, loss = 0.5385 
epoch 3/20,step 40/75, loss = 0.5537 
epoch 3/20,step 45/75, loss = 0.5571 
epoch 3/20,step 50/75, loss = 0.5979 
epoch 3/20,step 55/75, loss = 0.5615 
epoch 3/20,step 60/75, loss = 0.4850 
epoch 3/20,step 65/75, loss = 0.5517 
epoch 3/20,step 70/75, loss = 0.5433 
epoch 3/20,step 75/75, loss = 0.5660 
F1 score for training set is: 0.8384848086281731
Average train loss for epoch: 0.5415147646268209 


...

F1 score for validation set is: 0.8714547118023788
epoch 19/20,step 5/75, loss = 0.0086 
epoch 19/20,step 10/75, loss = 0.0049 
epoch 19/20,step 15/75, loss = 0.0034 
epoch 19/20,step 20/75, loss = 0.0028 
epoch 19/20,step 25/75, loss = 0.0097 
epoch 19/20,step 30/75, loss = 0.0152 
epoch 19/20,step 35/75, loss = 0.0028 
epoch 19/20,step 40/75, loss = 0.0031 
epoch 19/20,step 45/75, loss = 0.0028 
epoch 19/20,step 50/75, loss = 0.0021 
epoch 19/20,step 55/75, loss = 0.0126 
epoch 19/20,step 60/75, loss = 0.0230 
epoch 19/20,step 65/75, loss = 0.0089 
epoch 19/20,step 70/75, loss = 0.0036 
epoch 19/20,step 75/75, loss = 0.0060 
F1 score for training set is: 0.933867292874705
Average train loss for epoch: 0.010671736779622734 


100%|██████████| 15/15 [03:04<00:00, 12.33s/it]


F1 score for validation set is: 0.8562748305678898
epoch 20/20,step 5/75, loss = 0.0106 
epoch 20/20,step 10/75, loss = 0.0070 
epoch 20/20,step 15/75, loss = 0.0332 
epoch 20/20,step 20/75, loss = 0.2036 
epoch 20/20,step 25/75, loss = 0.0159 
epoch 20/20,step 30/75, loss = 0.0407 
epoch 20/20,step 35/75, loss = 0.0727 
epoch 20/20,step 40/75, loss = 0.0100 
epoch 20/20,step 45/75, loss = 0.0198 
epoch 20/20,step 50/75, loss = 0.0175 
epoch 20/20,step 55/75, loss = 0.0082 
epoch 20/20,step 60/75, loss = 0.0277 
epoch 20/20,step 65/75, loss = 0.0036 
epoch 20/20,step 70/75, loss = 0.0038 
epoch 20/20,step 75/75, loss = 0.0117 
F1 score for training set is: 0.9367469233507916
Average train loss for epoch: 0.029380885149973134 


100%|██████████| 15/15 [03:04<00:00, 12.33s/it]

F1 score for validation set is: 0.8764452505100883
